In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [61]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = 'B5tU2447OK8'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [63]:
node_prefix = 20
node_start_sym = 'a'
topics = [{
	"start": 17,
	"finish": 71 
},{
	"start": 71,
	"finish": 136 
},{
	"start": 136,
	"finish": 506
},{
	"start": 506,
	"finish": 545  
},{
	"start": 545,
	"finish": 637
},{
	"start": 637,
	"finish": 959
},{
	"start": 959,
	"finish": 1092  
},{
	"start": 1092,
	"finish": 1343
},{
	"start": 1343,
	"finish": 1418  
},{
	"start": 1418,
	"finish": 1478 
}]





In [65]:
import os

startNum = node_prefix
startChar = node_start_sym
path = '/Users/shota/github/content-ge/startup-school/nodes'


def make_folder(name):
    newNode = os.path.join(path, name)
    if not os.path.exists(newNode):
        os.makedirs(newNode)
    newFile = os.path.join(newNode, "text1.md")
    f = open(newFile, 'w')
    f.close()
    


for topic in topics:
    name = str(startNum) + startChar
    print('"' + name + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',')
    make_folder(name)
    startChar = chr(ord(startChar) + 1)


"20a": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "B5tU2447OK8",
            "start": 17,
            "end": 71
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"20b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "B5tU2447OK8",
            "start": 71,
            "end": 136
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"20c": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "B5tU2447OK8",
            "start": 136,
            "end": 506
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"20d": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],

In [ ]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }


startNum = node_prefix
startChar = node_start_sym

for topic in topics:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    name = str(startNum) + startChar
    newFile = os.path.join(os.path.join(path, name), "text1.md")
    f = open(newFile, 'w')
    print(resp.choices[0].message.content)
    f.write(resp.choices[0].message.content)
    f.close()
    startChar = chr(ord(startChar) + 1)
    print("====================================")

1. Topic Name: **Applying to and Succeeding at Y Combinator**
2. Topic Description: **Understanding the value of applying to Y Combinator and knowing the benefits and risk considerations.**

---

Alright, let's buckle up and explore the topic, **how to throw your hat in the ring for Y Combinator and make it work**. Now, you might be brimming with questions, like, why should I even bother applying to Y Combinator (YC)? Let me tell you, it's quite a worthy endeavor! And here's why:

Regardless of YC or not, anytime you can sit down and really consider your startup idea is golden. It's the kind of advice that's always valuable. The YC application process is fantastic for this because it asks questions that push you to straighten out and reflect on your ideas. Checking out these questions can help you understand your own business better and might even reveal some overlooked aspects or 'edge cases'.

If you're here learning about this, chances are YC could be a great fit for you. But, let's

1. Topic Name: Making a Strong Startup Application
2. Brief Description: Guidance on creating an effective application for a startup program.
3. Topic Rewrite:

Let's chat about what it feels to be on the other side - the person reviewing your startup application. Here are a few questions that often come up when I look at an application: 

1. **Who are the founders?**  It's vital to understand who is behind the startup – their history, how they met, and the roles each one plays in the startup.

2. **What's their idea?** This question seeks to understand what the startup is all about. Is it a unique idea? Is it easy to comprehend? What's the current product status?

3. **Is there any proof that people want this product?** Are there any users or customers who are eager for what you're building? Additionally, are the founders using the product themselves?

4. **Are the founders committed?** This could be illustrated by things like quitting a job to focus on the startup or applying multipl